In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
import seaborn as sns

from tqdm import tqdm_notebook
from IPython.display import display

import Levenshtein
import os

## The Dataset

My dataset is a patchwork collection of tables I found for my analysis goals with this project. Due to this there are some inconsistencies between the tables, that I have to deal with before any proper analysis can be done. 

- The [first table](https://data.world/craigkelly/steam-game-data) contains information about games on Steam. It is *very* in depth and contains many redundancies and unnecesary variables.
- The [second](https://www.kaggle.com/tamber/steam-video-games/home) contains playtime data for Steam users.
- The [third](https://www.kaggle.com/gregorut/videogamesales) is a collection of cummulative sales data (regional & global) for video games released on several platforms (not limited to PC only).



In [2]:
data_dir = './datasets/'

## Basic cleaning

The tables do not have alot of noise or missing values which need to be extensively preprocessed. The only main step executed here is filtering based on columns.

This table contains pretty much *all* the information available on a games Steam page. This contains many indicators which are not useful for the kinds of analysis I have in mind so I will pick and choose my columns rught here.

In [34]:
df_info = pd.read_csv(data_dir+'steam_game_info.csv', low_memory=False,
                      usecols=['ResponseName', 'ReleaseDate', 'RequiredAge', 'Metacritic', 'MovieCount', 'PackageCount', 
                               'RecommendationCount', 'SteamSpyOwners', 'SteamSpyOwnersVariance', 'SteamSpyPlayersEstimate', 
                               'SteamSpyPlayersVariance', 'AchievementCount', 'AchievementHighlightedCount', 'IsFree', 
                               'PlatformWindows', 'PlatformLinux', 'PlatformMac', 'CategorySinglePlayer', 'CategoryMultiplayer', 
                               'CategoryCoop', 'CategoryMMO', 'CategoryInAppPurchase', 'CategoryIncludeSrcSDK', 
                               'CategoryIncludeLevelEditor', 'CategoryVRSupport', 'GenreIsNonGame', 'GenreIsIndie', 'GenreIsAction', 
                               'GenreIsAdventure', 'GenreIsCasual', 'GenreIsStrategy', 'GenreIsRPG', 'GenreIsSimulation', 
                               'GenreIsEarlyAccess', 'GenreIsFreeToPlay', 'GenreIsSports', 'GenreIsRacing', 
                               'GenreIsMassivelyMultiplayer', 'PriceCurrency', 'PriceInitial', 'PriceFinal'])

print(f'{len(df_info["ResponseName"].unique())} unique games')
print(f'{df_info.isna().sum().sum()} missing values')

print(df_info.shape)
display(df_info.head())
df_info.to_csv(data_dir+'clean/steam_game_info.csv', index=False, header=True)

13199 unique games
0 missing values
(13357, 41)


,ResponseName,ReleaseDate,RequiredAge,Metacritic,MovieCount,PackageCount,RecommendationCount,SteamSpyOwners,SteamSpyOwnersVariance,SteamSpyPlayersEstimate,...,GenreIsRPG,GenreIsSimulation,GenreIsEarlyAccess,GenreIsFreeToPlay,GenreIsSports,GenreIsRacing,GenreIsMassivelyMultiplayer,PriceCurrency,PriceInitial,PriceFinal
0,Counter-Strike,Nov 1 2000,0,88,0,1,68991,13033334,92789,9140731,...,False,False,False,False,False,False,False,USD,9.99,9.99
1,Team Fortress Classic,Apr 1 1999,0,0,0,1,2439,5399140,60368,753627,...,False,False,False,False,False,False,False,USD,4.99,4.99
2,Day of Defeat,May 1 2003,0,79,0,1,2319,7621102,71499,1709740,...,False,False,False,False,False,False,False,USD,4.99,4.99
3,Deathmatch Classic,Jun 1 2001,0,0,0,1,888,7498965,70936,789676,...,False,False,False,False,False,False,False,USD,4.99,4.99
4,Half-Life: Opposing Force,Nov 1 1999,0,0,0,1,2934,5399499,60370,705741,...,False,False,False,False,False,False,False,USD,4.99,4.99


The sales data is split into two cleaned versions:

- One for PC only
- The other for all platforms (including PC)

In [50]:
df_sales = pd.read_csv(data_dir+'all_game_sales.csv')
df_sales = df_sales.drop(columns=['Rank'])

print(f'{len(df_sales["Name"].unique())} unique games')
print(f'{len(df_sales["Platform"].unique())} unique platforms')
print(f'{len(df_sales["Genre"].unique())} unique genres')
print(f'{len(df_sales["Publisher"].unique())} unique publishers')
      
print(df_sales.shape)
display(df_sales.head())
df_sales.to_csv(data_dir+'clean/all_game_sales.csv', index=False, header=True)

pconly = df_sales[df_sales['Platform'] == 'PC']
print(f'{len(pconly["Name"].unique())} unique PC games')
print(f'{pconly.isna().sum().sum()} missing values in PC data')

pconly.to_csv(data_dir+'clean/pc_game_sales.csv', index=False, header=True)      

11493 unique games
31 unique platforms
12 unique genres
579 unique publishers
(16598, 10)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


959 unique PC games
23 missing values in PC data


In [38]:
df_behav = pd.read_csv(data_dir+'steam_behaviour_info.csv', header=None, 
                      names=['user_id', 'game', 'type', 'hours', 'dummy'],
                      usecols=['user_id', 'game', 'type', 'hours'])

print(f'{len(df_behav["user_id"].unique())} unique users')
print(f'{len(df_behav["game"].unique())} unique games')
print(f'{df_behav.isna().sum().sum()} missing values')

df_behav = df_behav[df_behav['type'] == 'play']
df_behav = df_behav.drop(columns=['type'])

print(df_behav.shape)
display(df_behav.head())
df_behav.to_csv(data_dir+'clean/steam_behaviour_info.csv', index=False, header=True)

12393 unique users
5155 unique games
0 missing values
(70489, 3)


,user_id,game,hours
1,151603712,The Elder Scrolls V Skyrim,273.0
3,151603712,Fallout 4,87.0
5,151603712,Spore,14.9
7,151603712,Fallout New Vegas,12.1
9,151603712,Left 4 Dead 2,8.9
